In [78]:
import pandas as pd
import numpy as np
from plotnine import *

In [79]:
df = pd.read_csv("/Users/yu/Documents/Duke/courses/19fall/IDS690.02 python/mid-semester project/estimating-impact-of-opioid-prescription-regulations-team-4/20_intermediate_files/ultimate_final_frame_v2.csv")
df.head()

,Unnamed: 0,Year,FIPS,County,Morphine_equ,State,Deaths,Year_Changed,Population,Deaths_PC
0,0,2006.0,13001.0,APPLING,3996.843779,GA,NaN,NaN,17725.0,NaN
1,1,2006.0,13003.0,ATKINSON,1375.483686,GA,NaN,NaN,8197.0,NaN
2,2,2006.0,13005.0,BACON,4586.778211,GA,NaN,NaN,10682.0,NaN
3,3,2006.0,13007.0,BAKER,9.651600,GA,NaN,NaN,3877.0,NaN
4,4,2006.0,13009.0,BALDWIN,11069.705010,GA,NaN,NaN,46513.0,NaN


In [80]:
df_sub = df.drop(['Unnamed: 0','Morphine_equ','FIPS', 'Year_Changed'], axis = 1)
df_sub = df_sub.loc[(df_sub['State']=='TX') | (df_sub['State']=='CA') | (df_sub['State']=='NM') | (df_sub['State']=='AZ')].reset_index(drop = True)
df_sub.County.value_counts()
df_sub = df_sub.loc[df_sub['Deaths'].notnull()]
df_sub = df_sub.loc[df_sub['Population'].notnull()].reset_index(drop = True)
df_sub.Year.value_counts()

2010.0    97
2011.0    97
2012.0    92
2009.0    90
2008.0    88
2006.0    85
2007.0    84
Name: Year, dtype: int64

In [81]:
#check table is correct
overdose = pd.read_csv("/Users/yu/Documents/Duke/courses/19fall/IDS690.02 python/mid-semester project/estimating-impact-of-opioid-prescription-regulations-team-4/20_intermediate_files/overdose_deaths.csv")
overdose = overdose.drop(['index'], axis = 1)
overdose.Year.value_counts()

2015.0    794
2014.0    751
2013.0    706
2011.0    694
2012.0    682
2010.0    653
2009.0    599
2008.0    597
2007.0    572
2006.0    529
2005.0    481
2004.0    453
2003.0    409
Name: Year, dtype: int64

In [82]:
overdose_sub = overdose.loc[(overdose["State"] == 'TX') | (overdose["State"] == 'CA') | (overdose["State"] == 'NM') | (overdose["State"] == 'AZ')]
overdose_sub = overdose_sub.loc[(overdose_sub['Year'] >2005) & (overdose_sub['Year'] <2013)].reset_index(drop = True)
overdose_sub

,State,County,Year,Deaths
0,AZ,Cochise County,2006.0,13.0
1,AZ,Maricopa County,2006.0,601.0
2,AZ,Mohave County,2006.0,15.0
3,AZ,Navajo County,2006.0,10.0
4,AZ,Pima County,2006.0,190.0
...,...,...,...,...
628,TX,Travis County,2012.0,155.0
629,TX,Van Zandt County,2012.0,11.0
630,TX,Webb County,2012.0,29.0
631,TX,Wichita County,2012.0,18.0


In [83]:
df_sub.head()

,Year,County,State,Deaths,Population,Deaths_PC
0,2006.0,COCHISE,AZ,13.0,127241.0,0.000102
1,2006.0,MARICOPA,AZ,601.0,3642884.0,0.000165
2,2006.0,MOHAVE,AZ,15.0,196168.0,0.000076
3,2006.0,NAVAJO,AZ,10.0,105624.0,0.000095
4,2006.0,PIMA,AZ,190.0,940930.0,0.000202


In [84]:
#centered year by 2007
df_final = df_sub.copy()
df_final['Year'] = df_final['Year'] - 2007
df_final.head()

,Year,County,State,Deaths,Population,Deaths_PC
0,-1.0,COCHISE,AZ,13.0,127241.0,0.000102
1,-1.0,MARICOPA,AZ,601.0,3642884.0,0.000165
2,-1.0,MOHAVE,AZ,15.0,196168.0,0.000076
3,-1.0,NAVAJO,AZ,10.0,105624.0,0.000095
4,-1.0,PIMA,AZ,190.0,940930.0,0.000202


In [85]:
#create new table for analysis
overdose_inter = overdose.copy()
overdose_inter['County'] = overdose_inter['County'].replace(regex = True, to_replace = [' County', ' Borough', ' Parish', ' City'], value = '')
overdose_inter['County'] = overdose_inter['County'].replace(regex = True, to_replace = ['St. '], value = 'Saint ')
overdose_inter['County'] = overdose_inter['County'].str.upper()
overdose_inter['Year'] = overdose_inter['Year'].astype(int)
overdose_inter.head()

,State,County,Year,Deaths
0,AK,ANCHORAGE,2003,31.0
1,AK,MATANUSKA-SUSITNA,2003,11.0
2,AL,BALDWIN,2003,10.0
3,AL,JEFFERSON,2003,69.0
4,AL,MOBILE,2003,26.0


In [86]:
#filter final table again
df_inter = df.drop(['Unnamed: 0','Morphine_equ','FIPS', 'Year_Changed','Deaths'], axis = 1).copy()
df_inter.head()

,Year,County,State,Population,Deaths_PC
0,2006.0,APPLING,GA,17725.0,NaN
1,2006.0,ATKINSON,GA,8197.0,NaN
2,2006.0,BACON,GA,10682.0,NaN
3,2006.0,BAKER,GA,3877.0,NaN
4,2006.0,BALDWIN,GA,46513.0,NaN


In [89]:
myfinal = df_inter.merge(overdose_inter, how = 'outer', left_on = ['State','County','Year'], right_on = ['State','County','Year']).reset_index(drop = True)
myfinal.Year.value_counts()

2011.0    1437
2012.0    1430
2010.0    1403
2008.0    1359
2009.0    1358
2007.0    1351
2006.0    1316
2015.0     794
2014.0     751
2013.0     706
2005.0     481
2004.0     453
2003.0     409
Name: Year, dtype: int64

In [ ]:
#skip for now
#use population data of 2006 as population data for 2003-2005
#use population data of 2012 as population data for 2012-2015
#myfinal.loc[(myfinal["Year"] < 2006) & (myfinal["State"] == 'GA') & (myfinal["County"] == 'APPLING')] = \
#myfinal.loc[(myfinal["Year"] == 2006) & (myfinal["State"] == 'GA') & (myfinal["County"] == 'APPLING')]